In [38]:
from sklearn.datasets import load_iris
import numpy as np
import faiss, sklearn, pandas as pd
from scipy.optimize import minimize
import matplotlib.pyplot as plt
%matplotlib widget
from IPython.core.display import HTML

First we import the functions required to (a) predict the mass $\hat{m}$ an uncertain point (b) calculate the evidential log likelihood and (c) calculate the gradient of the evidential loglikelihood with respect to the decays parameters.

The Frame of Discernment is composed of $\{F,NF, \Theta\}$ (Fraud, non-fraud and "uncertain" between the two)

Each mass in the neighborhood is discounted according to: $m'_j(F) = m_j(F) \alpha e^{- \gamma_1 d^2_{ij}}$ and $m'_j(NF) = m_j(NF) \alpha e^{- \gamma_2 d^2_{ij}}$ and then pooled according to $$\hat{m} = \oplus_{x_i \in \mathcal{N}_k(x)} m'_i$$ where $\oplus$ is the dempster rule of combination and $m_j$ the respective masses of observations $x_j$ in the neighborhood $\mathcal{N}_k$ of a new observation $x$.

Via the gradient we try to estimate the "best" values for the discount parameters $(\alpha, \gamma_1,\gamma_2)$

In [39]:
# The functions
def Gradient(decays, I, D, pl_train, pl_test):
    # Algorithm 1. Gradient of the loglikelihood with respect to decays parameters, cf Denoeux 2019.
    dists = D ** 2
    k = I.shape[1]
    alpha = 1 / (1 + np.exp(decays[0]))
    gamma = decays[1:] ** 2
    beta = alpha * (np.exp(-gamma.reshape(1, 1, 2) * dists.reshape(dists.shape[0], dists.shape[1], 1)))
    discounted_pl = 1 - beta + (beta * pl_train[I])
    hat_pl = discounted_pl[:, 0, :]
    for n in range(1, k - 1):
        hat_pl = hat_pl * discounted_pl[:, n, :]
    norm_hat_pl = hat_pl / (hat_pl.sum(axis=1).reshape(hat_pl.shape[0], 1))

    # gradient calculation cf Algorithm 3 Denoeux et al. 2019
    a4 = pl_test / np.expand_dims((norm_hat_pl * pl_test).sum(axis=1), axis=1)  # A.4 (i,q)
    a8 = (np.expand_dims(-hat_pl, axis=1) * (1 - pl_train[I])) / (
            1 - (beta * (1 - pl_train[I])))  # A.8 (i,K,q) !!! not ok check denominator
    a9 = - beta * np.expand_dims(dists, axis=2)  # A.9 (i,K,q)
    a13 = beta / alpha  # A.13 (i,K,q)
    a7 = (a8 * a9).sum(axis=1)  # A.7 (i,q)
    a12 = (a8 * a13).sum(axis=1)  # A.12 (i,q)
    a6_1 = (1 - norm_hat_pl) / (hat_pl.sum(axis=1).reshape(hat_pl.shape[0], 1))  # (i,q=0,k=0) & (i,q=1,k=1)
    a6_2 = -hat_pl / (hat_pl.sum(axis=1).reshape(hat_pl.shape[0], 1) ** 2)  # (i,q=0,k=1) & (i,q=1,k=0)
    a6 = np.stack((np.stack((a6_1[:, 0], a6_2[:, 0]), axis=1),
                   np.stack((a6_2[:, 1], a6_1[:, 1]), axis=1)),
                  axis=2)  # (i,k,q)
    a5 = a6 * np.expand_dims(a7, axis=2)  # (i,k,q)
    a3 = 2 * np.expand_dims(decays[1:], axis=0) * (np.expand_dims(a4, axis=1) * a5).sum(
        axis=2)  # (i,k) # why is second term symetric ?
    a11 = (a6 * np.expand_dims(a12, axis=2)).sum(axis=1)
    a10 = alpha * (1 - alpha) * (a4 * a11).sum(axis=1)
    a1 = a3.sum(axis=0)
    a2 = a10.sum(axis=0)
    return np.concatenate((np.array([a2]),a1))
def eloglik(decays, I, D , pl_train, pl_test):
    # Algorithm 1. Calculation of the evidential likelihood, cf Denoeux 2019.
    dists = D ** 2 # distance are squared
    k = I.shape[1]
    alpha = 1 / (1 + np.exp(decays[0])) # variable transformation for optimisation cf Denoeux 2019
    gamma = decays[1:] ** 2 # variable transformation for optimisation cf Denoeux 2019
    beta = alpha * (np.exp(-gamma.reshape(1, 1, 2) * dists.reshape(dists.shape[0], dists.shape[1], 1)))
    discounted_pl = 1 - beta + (beta * pl_train[I]) # neighbors' plausibilities discounted
    hat_pl = discounted_pl[:, 0, :] # first neighbor's plausibility
    for n in range(1, k - 1): # aggregate over neighbor 
        hat_pl = hat_pl * discounted_pl[:, n, :]
    norm_hat_pl = hat_pl / (hat_pl.sum(axis=1).reshape(hat_pl.shape[0], 1))
    # auc if needed
    # fpr, tpr, thresholds = sklearn.metrics.roc_curve(np.round(pl_test)[:,0], norm_hat_pl[:,0])
    # print(sklearn.metrics.auc(fpr, tpr))
    return -np.sum(np.log((norm_hat_pl * pl_test).sum(axis=1)))
def predict_eknn(decays, I, D, my):
    # Aggregation of masses over a (weighted, or discounted) neighborhood
    nb = I
    dists = D ** 2 #preds[0]
    k = nb.shape[1]
    alpha = 1 / (1 + np.exp(decays[0]))
    gamma = decays[1:] ** 2
    m0 = my[nb][:,:,0] * alpha * (np.exp(-gamma[0] * dists)) #* (np.exp(-time_decay * times))
    m1 = my[nb][:,:,1] * alpha * (np.exp(-gamma[1] * dists)) #* (np.exp(-time_decay * times))
    m10 = 1 - m0 - m1
    m0t = m0[:,0]
    m1t = m1[:,0]
    m10t = m10[:,0]
    for n in range(k-1):
        denominators = 1 - (m0t * m1[:,n+1] + m0[:,n+1] * m1t) # denominators
        m0t = (m0t * m10[:,n+1] + m0[:,n+1] * m10t + m0t * m0[:,n+1])/denominators
        m1t = (m1t * m10[:,n+1] + m1[:,n+1] * m10t + m1t * m1[:,n+1])/denominators
        m10t = 1 - m0t - m1t
    return np.vstack((m0t,m1t,m10t)).T

We load the iris dataset and subset it to focus on two classes only.
We also artficially introduce uncertainty on the labels to mimic imperfect labels (e.g. each flower has been observed with a certain degree of reliability).

$m(\{\Theta\})$ is the vacuous function and represents total uncertainty
$m(\{\theta_0\})$ and
$m(\{\theta_1\})$ are the masses of respectively class $\theta_0$ (e.g., Fraud) and class $\theta_1$ (e.g., Not Fraud).

Each point in the training set $\mathcal{T} = \{ (x_i,m_i), i = 1,...,n\}$ is assigned a mass $m_i(\{\theta_0\})$, $m_i(\{\theta_1\})$, $m_i(\{\Theta\})$ as follows:

In [40]:
X, y = load_iris().data[1:100, [0, 2]], load_iris().target[1:100]
# represents m(\theta_0), m(\theta_1) and m(\Theta)
my = np.zeros(shape=(y.size,3))
my[:,0] = np.where(y == 0,1,0)
my[:,1] = np.where(y == 1,1,0)
my = (my.T * np.random.beta(5,1,my.shape[0])).T #add some uncertainty on the labels at random here, just to show how it would work with uncertain labels.
my[:,2] = 1 - my[:,1] - my[:,0]
pl = my[:, :-1] + my[:, -1].reshape(my.shape[0], 1)

print(pd.DataFrame(my[45:55,:], columns=["$m(\theta_0)$", "$m(\theta_1)$", "$m(\Theta)$"]))

   $m(\theta_0)$  $m(\theta_1)$  $m(\Theta)$
0       0.791721       0.000000     0.208279
1       0.894184       0.000000     0.105816
2       0.871428       0.000000     0.128572
3       0.825931       0.000000     0.174069
4       0.000000       0.923634     0.076366
5       0.000000       0.913915     0.086085
6       0.000000       0.894069     0.105931
7       0.000000       0.878683     0.121317
8       0.000000       0.759366     0.240634
9       0.000000       0.930676     0.069324


We can visualise our dataset, the color gradients are just there to represent the uncertainty of each point. 
We can clearly identify two groups nevertheless

In [41]:
import matplotlib.pyplot as plt
plt.scatter(X[:,0], X[:,1],c=(my[:,0]+my[:,2]/2), edgecolor='k', s=20)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Next we divide our dataset in a train and test sets, we calculate the neighbors of each observation in the test set.

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, my_train, my_test, pl_train, pl_test = train_test_split(X,my,pl, test_size=0.2)
index = faiss.IndexFlatL2(X_train.shape[1]) # In this case we use euclidean distance.
index.add(np.ascontiguousarray(X_train, dtype=np.float32))  # should convert once to arrays np.float out of loop.
D, I = index.search(np.ascontiguousarray(X_test, dtype=np.float32), k=5) # here k=5 neighbors, can be changed

Here we use Broyden–Fletcher–Goldfarb–Shanno method, a numerical method for numerical optimisation. 
Note that the gradient is used at this stage and we learn the parameters $(\alpha, \gamma_0, \gamma_1)$. In this example we have very little information in the test set, so it might be unstable here.

In [43]:
res = minimize(eloglik, np.array((0, 1, 1)), method='BFGS', jac=Gradient,
               options={'disp': True}, args=(I, D, pl_train, pl_test))

         Current function value: 0.005858
         Iterations: 12
         Function evaluations: 77
         Gradient evaluations: 65


In [44]:
res.x

array([-6.47131808,  1.68339459,  1.12055593])

Finally we can visualize the results:

In [45]:
def predict_eknn_forgrid(X_test = [6, 4.5], index=index, resx = res.x, my = my_train):
    D, I = index.search(np.ascontiguousarray(X_test, dtype=np.float32).reshape(1,2), k=5)
    result = predict_eknn(resx, I=I, D=D, my=my)
    return(result)

x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
h = .05
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
grid_to_assess = np.c_[xx.ravel(), yy.ravel()]

Z = np.apply_along_axis(predict_eknn_forgrid, 1, grid_to_assess)

m1 = Z[:,0,0].ravel().reshape(xx.shape)
m2 = Z[:,0,1].ravel().reshape(xx.shape)
m3 = Z[:,0,2].ravel().reshape(xx.shape)

from matplotlib import cm
import matplotlib.pyplot as plt
fig, (ax1,ax2,ax3) = plt.subplots(1,3, subplot_kw={"projection": "3d"})
ax1.set_title(r'$m(\theta_0)$')
ax1.plot_surface(xx, yy, m1,cmap=cm.coolwarm,linewidth=0, antialiased=False,alpha=.5, label = 'mass')
ax1.scatter(X[:,0], X[:,1],c=(my[:,0]+my[:,2]/2), label='training points')
ax2.set_title(r'$m(\theta_1)$')
ax2.plot_surface(xx, yy, m2,cmap=cm.coolwarm,linewidth=0, antialiased=False,alpha=.5, label = 'mass')
ax2.scatter(X[:,0], X[:,1],c=(my[:,0]+my[:,2]/2), label='training points')
ax3.set_title(r'$m(\Theta)$ Uncertainy')
ax3.plot_surface(xx, yy, m3,cmap=cm.coolwarm,linewidth=0, antialiased=False,alpha=.5, label = 'mass')
ax3.scatter(X[:,0], X[:,1],c=(my[:,0]+my[:,2]/2), label='training points')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …